In [1]:
import numpy as np
import pandas as pd
import wandb
import torch
from pathlib import Path
from tqdm import tqdm


In [2]:
mapping = {
    'test/mean_recall_at_10': "R@10",
    'test/mean_recall_at_5': "R@5",
    'test/mean_recall_at_3': "R@3",
    'test/mean_top1_distance': "mean_top1_distance",
    'test/mean_recall_at_1%': "R@1%",
    'test/mean_recall_at_1': "R@1",
}


In [56]:
metrics_df = pd.DataFrame(columns=['dataset', "modality", 'exp_name', 'R@1', 'R@3', 'R@5', 'R@10', 'R@1%', 'mean_top1_distance'])

api = wandb.Api()

# Replace "your_project_name" with the name of your project
runs = api.runs("CVPR2024")

for run in tqdm(runs, leave=False):
    if run.state == 'finished':
        if run.name.endswith("single_camera"):
            run.name = run.name.replace("single_camera", "camera1")
        if run.name.endswith("semantic"):
            run.name = run.name + "1"
        run_name = run.name
        if run_name.startswith("convnext"):
            continue  # Skip ConvNeXt runs
        dataset, modality = run_name.split('_')[0], run_name.split('_')[1]
        exp_name = "_".join(run_name.split('_')[2:]) if len(run_name.split('_')) > 2 else "exp"

        # Get the history of the "test/mean_recall_at_1" metric
        history = run.history()

        # Find the step number for the biggest value
        best_epoch = np.argmax(history["test/mean_recall_at_1"])
        best_metrics = history.iloc[best_epoch]
        best_test_metrics = best_metrics.filter(regex='^test/')
        best_test_metrics = best_test_metrics.rename(index=mapping)
        best_test_metrics['dataset'] = dataset
        best_test_metrics['modality'] = modality
        best_test_metrics['exp_name'] = exp_name

        # Download the best checkpoint
        checkpoint_wandb = run.file("best.pth")
        checkpoint_wandb.name = f"{dataset}_{modality}_{exp_name}.pth"
        if not Path(f"./checkpoints/{checkpoint_wandb.name}").exists():
            checkpoint_wandb.download(root="./checkpoints/", replace=False, exist_ok=False)
            checkpoint = torch.load(f"./checkpoints/{checkpoint_wandb.name}", map_location="cpu")
            checkpoint["config"] = run.config
            torch.save(checkpoint, f"./checkpoints/{checkpoint_wandb.name}")

        metrics_df = pd.concat([metrics_df, best_test_metrics.to_frame().T], ignore_index=True)

metrics_df.sort_values(by=['dataset', 'modality', 'exp_name'], inplace=True, ignore_index=True)
metrics_df


,dataset,modality,exp_name,R@1,R@3,R@5,R@10,R@1%,mean_top1_distance
0,nclt,camera1,exp,0.66,0.78,0.82,0.87,0.77,0.47
1,nclt,camera2-front-back,add,0.75,0.86,0.90,0.94,0.86,0.62
2,nclt,camera2-front-back,concat,0.72,0.81,0.84,0.89,0.80,0.65
3,nclt,camera2-front-back,mlp-full,0.63,0.79,0.85,0.91,0.78,0.34
4,nclt,camera2-front-back,mlp-half,0.60,0.78,0.84,0.90,0.76,0.29
...,...,...,...,...,...,...,...,...,...
99,oxford,text4,clip-base-mlp-concat,0.10,0.22,0.30,0.44,0.24,0.06
100,oxford,text4,clip-large-mlp-add,0.09,0.21,0.30,0.44,0.23,0.62
101,oxford,text4,clip-large-mlp-concat,0.11,0.24,0.32,0.46,0.25,0.33
102,oxford,text4,tfidf-mlp-add,0.07,0.17,0.24,0.38,0.18,0.00


In [57]:
metrics_df.to_csv("metrics.csv", index=False)


In [3]:
metrics_df = pd.read_csv("metrics.csv")


In [59]:
metrics_df


,dataset,modality,exp_name,R@1,R@3,R@5,R@10,R@1%,mean_top1_distance
0,nclt,camera1,exp,0.66,0.78,0.82,0.87,0.77,0.47
1,nclt,camera2-front-back,add,0.75,0.86,0.90,0.94,0.86,0.62
2,nclt,camera2-front-back,concat,0.72,0.81,0.84,0.89,0.80,0.65
3,nclt,camera2-front-back,mlp-full,0.63,0.79,0.85,0.91,0.78,0.34
4,nclt,camera2-front-back,mlp-half,0.60,0.78,0.84,0.90,0.76,0.29
...,...,...,...,...,...,...,...,...,...
99,oxford,text4,clip-base-mlp-concat,0.10,0.22,0.30,0.44,0.24,0.06
100,oxford,text4,clip-large-mlp-add,0.09,0.21,0.30,0.44,0.23,0.62
101,oxford,text4,clip-large-mlp-concat,0.11,0.24,0.32,0.46,0.25,0.33
102,oxford,text4,tfidf-mlp-add,0.07,0.17,0.24,0.38,0.18,0.00
